In [15]:
import shutil
import os
import random

import numpy as np

In [16]:
# preprocssed dataset path
DATASET_PATH = os.path.abspath(os.path.join("..", "storage", "datasets", "cleaned"))

# actions to be detected
ACTIONS = [
       "tolong", "maaf", "sama-sama"
]

# number of videos and actions per video

# current dataset have 120 video, so technically:
# 120 for processed image (not flipped horizontally)
# 120 for processed image (flipped horizontally)
videos_per_label = 120

# 60 frame per video
frames_per_video = 60

# data labels
labels_map = {label: index for index, label in enumerate(ACTIONS)}

In [17]:
def add_noise(landmarks, noise_level, shape=(225,)):
    noise = np.random.normal(loc=0, scale=noise_level, size=shape)
    return landmarks + noise


def shift_landmarks(landmarks, shift_factor, shape=(225,)):
    shift = np.random.uniform(-shift_factor, shift_factor, size=shape[1:])
    return landmarks + shift


def augment_landmarks(landmarks, noise_level=0.01, shift_factor=0.1):
    if random.random() > 0.6:
        landmarks = add_noise(landmarks, noise_level)

    if random.random() > 0.6:
        landmarks = shift_landmarks(landmarks, shift_factor)

    return landmarks

In [18]:
seq_path = os.path.abspath("d:/sds-ml-main/")
lab_path = os.path.abspath("d:/sds-ml-main/")

In [19]:
def generate_bulk_dataset_and_load(augment=True):
    for action in ACTIONS:
        sequences, labels = [], []

        for sequence in range(videos_per_label):
            # construct the path to the numpy file for the current frame
            npy_path = os.path.join(DATASET_PATH, action, f"{sequence}.npy")
            
            #if npy doesn't exist
            if not os.path.exists(npy_path):
                print(f"Warning: {npy_path} does not exist.")
                continue

            # load the frame data from the numpy file
            result = np.load(npy_path)

            # append the completed sequence to the sequences list
            sequences.append(result)

            # append the corresponding label to the labels list
            labels.append(labels_map[action])

            if not augment:
                continue

            # number of augmented sequences to create per original sequence
            num_augmented_sequences = int(0.04 * videos_per_label)  # 4% of total videos

            for _ in range(num_augmented_sequences):
                augmented_sequence = [
                    augment_landmarks(frame, noise_level=0.078, shift_factor=0.65)
                    for frame in result
                ]

                # append the augmented sequence to the sequences list
                sequences.append(augmented_sequence)

                # append the corresponding label to the labels list
                labels.append(labels_map[action])

        # convert to numpy arrays
        sequences_np = np.array(sequences)
        labels_np = np.array(labels)

        try:
            shutil.rmtree(f"{seq_path}/{action}")
        except:
            os.makedirs(f"{seq_path}/{action}")


        # save the sequences and labels for the current action
        seq_dir = os.path.join(seq_path, f"{action}_sequences.npy")
        lab_dir = os.path.join(lab_path, f"{action}_labels.npy")

        print(f"Sequences \"{action}\" Saved")
        print("="*50)
        np.save(seq_dir, sequences_np)
        np.save(lab_dir, labels_np)

In [20]:
generate_bulk_dataset_and_load(augment=True)

Sequences "tolong" Saved
Sequences "maaf" Saved
Sequences "sama-sama" Saved


In [21]:
def load_and_combine_datasets(seq_dir, lab_dir):
    all_sequences = []
    all_labels = []

    for action in ACTIONS:
        seq_path = os.path.join(seq_dir, f"{action}_sequences.npy")
        lab_path = os.path.join(lab_dir, f"{action}_labels.npy")

        # load sequences and labels
        sequences = np.load(seq_path)
        labels = np.load(lab_path)

        all_sequences.append(sequences)
        all_labels.append(labels)

    # combine all sequences and labels into single arrays
    combined_sequences = np.concatenate(all_sequences, axis=0)
    combined_labels = np.concatenate(all_labels, axis=0)

    return combined_sequences, combined_labels

In [22]:
# load and combine the dataset
sequences, labels = load_and_combine_datasets(seq_path, lab_path)

In [23]:
# save the combined dataset
np.save(os.path.join("d:/sds-ml-main/", "combined_sequences.npy"), sequences)
np.save(os.path.join("d:/sds-ml-main/", "combined_labels.npy"), labels)

In [24]:
# please run this

import gc # garbage collector

del sequences
del labels

gc.collect()

284